# 导入库

In [1]:
#安装相关依赖库 如果是windows系统，cmd命令框中输入pip安装，或在Jupyter notebook中!pip安装，参考上述环境配置
#!pip install pandas numpy cv2 torch torchvision time albumentations timm tqdm

#---------------------------------------------------
#导入库
import cv2
import time
import pandas as pd
import numpy as np
from tqdm import tqdm
import torch.nn.functional as F
import torch
import torch.nn as nn
from torch.utils.data.dataset import Dataset
from torchvision import models
from torchvision.datasets import DatasetFolder

import timm

C:\Users\BiXY\anaconda3\envs\mmcv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import albumentations as A

# 框架设置

In [3]:
#----------------框架设置----------------
#设置torch使用gpu
torch.manual_seed(0)
torch.backends.cudnn.deterministic = False
torch.backends.cudnn.benchmark = True
device = 'cuda' if torch.cuda.is_available() else 'cpu'
#输出cuda说明使用gpu，输出cpu说明使用cpu，最好使用gpu训练
print(device)

cuda


# 数据处理

In [4]:
#----------------数据处理----------------
#基础数据读取
train_df = pd.read_csv('data/train.csv')
train_df['path'] = 'data/train/' + train_df['image']


test_df = pd.read_csv('data/提交示例.csv')
test_df['path'] = 'data/test/' + test_df['image'] 

# 定义数据集读取方法
class XunFeiDataset(Dataset):
    def __init__(self, img_path, label, transform=None):        
        self.img_path = img_path        
        self.label = label        
        if transform is not None:           
            self.transform = transform        
        else:            
            self.transform = None        
    def __getitem__(self, index):        
        img = cv2.imread(self.img_path[index])                    
        img = img.astype(np.float32)                
        img /= 255.0        
        img -= 1                
        
        if self.transform is not None:            
            img = self.transform(image = img)['image']        
        img = img.transpose([2,0,1])                
        
        return img,torch.from_numpy(np.array(self.label[index]))        
    
    def __len__(self):        
        return len(self.img_path)
   
   
#使用torch批量数据读取
train_loader = torch.utils.data.DataLoader(    
    XunFeiDataset(train_df['path'].values, train_df['label'].values,           
    A.Compose([            
        A.RandomCrop(450, 750),
        A.CoarseDropout(p=0.5)
        ])
        ), batch_size=8, shuffle=True, num_workers=0, pin_memory=False)
    
val_loader = torch.utils.data.DataLoader(    
    XunFeiDataset(train_df['path'].values[:-200], train_df['label'].values[:-200],            
    A.Compose([            
        A.RandomCrop(450, 750), 
        A.CoarseDropout(p=0.5)
        ])    
        ), batch_size=2, shuffle=False, num_workers=0, pin_memory=False)    

In [5]:
class LRScheduler():
    """
    Learning rate scheduler. If the validation loss does not decrease for the
    given number of `patience` epochs, then the learning rate will decrease by
    by given `factor`.
    """
    def __init__(
        self, optimizer, patience=5, min_lr=1e-6, factor=0.5
    ):
        """
        new_lr = old_lr * factor
        :param optimizer: the optimizer we are using
        :param patience: how many epochs to wait before updating the lr
        :param min_lr: least lr value to reduce to while updating
        :param factor: factor by which the lr should be updated
        """
        self.optimizer = optimizer
        self.patience = patience
        self.min_lr = min_lr
        self.factor = factor
        self.lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
                self.optimizer,
                mode='min',
                patience=self.patience,
                factor=self.factor,
                min_lr=self.min_lr,
                verbose=True
            )
    def __call__(self, val_loss):
        self.lr_scheduler.step(val_loss)

class EarlyStopping():
    """
    Early stopping to stop the training when the loss does not improve after
    certain epochs.
    """
    def __init__(self, patience=5, min_delta=0):
        """
        :param patience: how many epochs to wait before stopping when loss is
               not improving
        :param min_delta: minimum difference between new loss and old loss for
               new loss to be considered as an improvement
        """
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_loss = None
        self.early_stop = False
    def __call__(self, val_loss):
        if self.best_loss == None:
            self.best_loss = val_loss
        elif self.best_loss - val_loss > self.min_delta:
            self.best_loss = val_loss
            # reset counter if validation loss improves
            self.counter = 0
        elif self.best_loss - val_loss < self.min_delta:
            self.counter += 1
            print(f"INFO: Early stopping counter {self.counter} of {self.patience}")
            if self.counter >= self.patience:
                print('INFO: Early stopping')
                self.early_stop = True

# 模型训练

In [6]:
       
       
#----------------模型训练----------------
       
# 模型训练一个epoch的函数
def train(train_loader, model, criterion, optimizer):   
    model.train()    
    train_loss = 0.0    
    
    for i, (input, target) in enumerate(train_loader):        
        input = input.to(device)        
        target = target.to(device)        
        output = model(input)        
        loss = criterion(output, target)        
        optimizer.zero_grad()        
        loss.backward()        
        optimizer.step()
        if i % 40 == 0:            
            print(loss.item())                    
            
        train_loss += loss.item()        
    
    return train_loss/len(train_loader)

# 模型验证一个epoch的函数
def validate(val_loader, model, criterion):    
    model.eval()
    val_acc = 0.0
    val_running_loss = 0.0
    counter = 0
    with torch.no_grad():        
        end = time.time()        
        for i, (input, target) in enumerate(val_loader): 
            counter += 1
            input = input.to(device)            
            target = target.to(device)            
            output = model(input)            
            # loss = criterion(output, target)
            val_acc += (output.argmax(1) == target).sum().item()                
            loss = criterion(output, target)
            val_running_loss += loss.item()
        val_loss = val_running_loss / counter
        return val_acc / len(val_loader.dataset), val_loss    
        
# 模型预测函数     
def predict(test_loader, model, criterion):    
    model.eval()    
    val_acc = 0.0        
    
    test_pred = []    
    with torch.no_grad():        
        end = time.time()        
        for i, (input, target) in enumerate(test_loader):    
            input = input.to(device)            
            target = target.to(device)            
            output = model(input)            
            test_pred.append(output.data.cpu().numpy())                
            return np.vstack(test_pred)

class LabelSmoothing(nn.Module):
    """NLL loss with label smoothing.
    """
    def __init__(self, smoothing=0.0):
        """Constructor for the LabelSmoothing module.
        :param smoothing: label smoothing factor
        """
        super(LabelSmoothing, self).__init__()
        self.confidence = 1.0 - smoothing
        self.smoothing = smoothing
        # 此处的self.smoothing即我们的epsilon平滑参数。

    def forward(self, x, target):
        logprobs = torch.nn.functional.log_softmax(x, dim=-1)
        nll_loss = -logprobs.gather(dim=-1, index=target.unsqueeze(1))
        nll_loss = nll_loss.squeeze(1)
        smooth_loss = -logprobs.mean(dim=-1)
        loss = self.confidence * nll_loss + self.smoothing * smooth_loss
        return loss.mean()

# 定义模型，使用resnet18
print("Creating model----{}".format('resnet18d'))
model = timm.create_model('resnet18d', pretrained=True, num_classes=24)  # 通过修改模型名字更换不同模型  
model = model.to(device)
# criterion = nn.CrossEntropyLoss()
criterion2 = LabelSmoothing(smoothing=0.1)
optimizer = torch.optim.Adam(model.parameters(), 0.0001)
lr_scheduler = LRScheduler(optimizer)
early_stopping = EarlyStopping()

val_loss_s = []
# 模型训练
#resnet18大概消耗显存：3.7GB，使用Gtx1060时，训练每个epoch花费大概2min
for i in range(25):
    print("------epoch{}----------".format(i))   
    print("-------Loss----------")      
#     train_loss = train(train_loader, model, criterion, optimizer)
    train_loss2 = train(train_loader, model, criterion2, optimizer)
#     print(train_loss)
    print(train_loss2)
    
    
    print("-------Val acc----------") 
#     val_acc = validate(val_loader, model, criterion)  
    val_acc2, val_loss  = validate(val_loader, model, criterion2)   
    val_loss_s.append(val_loss)
    lr_scheduler(val_loss)
    early_stopping(val_loss)
    if early_stopping.early_stop:
        break
#     print(val_acc)
    print(val_acc2)   
    
    
# 模型预测
test_loader = torch.utils.data.DataLoader(    
    XunFeiDataset(test_df['path'].values, [0] * test_df.shape[0],            
        A.Compose([            
            A.RandomCrop(450, 750), 
            ])    
            ), batch_size=2, shuffle=False, num_workers=0, pin_memory=False)

model.eval()    
val_acc = 0.0        


test_pred = []    
with torch.no_grad():  
    for input, _ in tqdm(test_loader):
        # print(img[0])
        input = input.to(device)                      
        output = model(input)            
        test_pred.append(output.data.cpu().numpy())                

pred = np.vstack(test_pred)



Creating model----resnet18d
------epoch0----------
-------Loss----------
3.127227783203125
3.1971163749694824
3.153848886489868
3.159370183944702
3.1751434803009033
2.9723458290100098
2.9698309898376465
3.1253415639720745
-------Val acc----------
0.27380339680905813
------epoch1----------
-------Loss----------
2.791243553161621
2.9250807762145996
2.6188745498657227
2.701113700866699
2.3806347846984863
2.249156951904297
2.282310724258423
2.5663539971878278
-------Val acc----------
0.7344312918167781
------epoch2----------
-------Loss----------
2.3944058418273926
2.4326140880584717
2.3966429233551025
1.9910833835601807
2.175626754760742
2.183783769607544
1.935177206993103
2.099251750245023
-------Val acc----------
0.9047864127637674
------epoch3----------
-------Loss----------
1.8523657321929932
1.8172318935394287
1.8241829872131348
1.96187424659729
1.9777195453643799
1.8914158344268799
1.9881277084350586
1.892679771380638
-------Val acc----------
0.9397838394235718
------epoch4---------

100%|████████████████████████████████████████████████████████████████████████████████| 510/510 [00:19<00:00, 26.36it/s]


In [7]:
#----------------结果输出----------------
pd.DataFrame(    
    {        
        'image': [x.split('/')[-1] for x in test_df['path'].values],        
        'label': pred.argmax(1)
        }).to_csv('result.csv', index=None)

In [ ]:
# 随机擦除
def random_erase(img,n_holes,length,rate): #输入img为PIL图片格式的图片
    if np.random.rand(1)[0]<rate:
        img = np.array(img)
        h = img.shape[0] #图片的高
        w = img.shape[1] #图片的宽
        
        n_holes = np.random.randint(n_holes)
        mask = np.ones((h, w), np.float32) #32*32w*h的全1矩阵

        for n in range(n_holes): #n_holes=2,length=4 选择2个区域；每个区域的边长为4
            y = np.random.randint(h) #0~31随机选择一个数 y=4
            x = np.random.randint(w) #0~31随机选择一个数 x=24

            y1 = np.clip(y - length // 2, 0, h) #2,0,32 ->2
            y2 = np.clip(y + length // 2, 0, h) #6,0,32 ->6
            x1 = np.clip(x - length // 2, 0, w) #24-2,0,32 ->22
            x2 = np.clip(x + length // 2, 0, w) #24+2,0,32 ->26

            mask[y1: y2, x1: x2] = 0. #将这一小块区域去除
        img[:,:,0] = img[:,:,0] * mask
        img[:,:,1] = img[:,:,1] * mask
        img[:,:,2] = img[:,:,2] * mask
        return Image.fromarray(img)
    else:
        return img

In [ ]:
def random_mixup(img ,label, mixup_img, mixup_label):#输入img和mixup为IMG格式的图片，label和mixup_label为int类型
    img = np.array(img)
    mixup_img = np.array(mixup_img)
    label_onehot = np.zeros(24)
    label_onehot[label] = 1
    mixup_label_onehot = np.zeros(24)
    mixup_label_onehot[mixup_label] = 1

    alpha = 1
    lam = np.random.beta(alpha,alpha) #混合比例

    img_new = lam*img + (1-lam)*mixup_img
    label_new = lam*label_onehot + (1-lam)*mixup_label_onehot
    
    return Image.fromarray(np.uint8(img_new)), torch.to_tensor(np.float32(label_new))

In [ ]:
def getStat(train_data):
    '''
    Compute mean and variance for training data
    :param train_data: 自定义类Dataset(或ImageFolder即可)
    :return: (mean, std)
    '''
    print('Compute mean and variance for training data.')
    print(len(train_data))
    train_loader = DataLoader(
        train_data, batch_size=1, shuffle=False, num_workers=0)
    mean = np.zeros(3)
    std = np.zeros(3)
    for X, _ in train_loader:
        for d in range(3):
            mean[d] += X[:, d, :, :].mean().cpu().numpy()[0]
            std[d] += X[:, d, :, :].std().cpu().numpy()[0]
    mean = mean/len(train_data)
    std = std/len(train_data)
    return list(mean), list(std)

In [ ]:
# 定义数据集读取方法
class XunFeiDataset(Dataset):
    def __init__(self, img_path, label, transforms=None, mode='train'):        
        self.img_path = img_path        
        self.label = label   
        self.transforms = transforms 
        self.mode = mode           
    def __getitem__(self, index):        
        img = Image.open(self.img_path[index]).convert('RGB') 

        #将label转化为one_hot编码
        label_onehot = np.zeros(24)
        label_onehot[self.label[index]] = 1
        label_onehot = torch.to_tensor(np.float32(label_onehot))   

        if self.mode == 'train': #训练时才做数据增强
            #随机擦除 100代表100个正方形，10代表每个正方形边长为10,0.2代表20%的概率
            img = random_erase(img,100,10,0.2) 
            #mixup,0.2的概率
            if np.random.rand(1)[0]<0.2:
                mixup_idx = np.random.randint(0, len(self.img_path)-1)
                mixup_img = Image.open(self.img_path[mixup_idx]).convert('RGB')
                mixup_label = self.label[mixup_idx]
                img, label_onehot = random_mixup(img, self.label[index], mixup_img, mixup_label)
            #cutmix,0.2的概率
            if np.random.rand(1)[0]<0.2:
                cutmix_idx = np.random.randint(0, len(self.img_path)-1)
                cutmix_img = Image.open(self.img_path[cutmix_idx]).convert('RGB')
                cutmix_label = self.label[cutmix_idx]
                img, label_onehot = cutmix(img, self.label[index], cutmix_img, cutmix_label)

        if self.transforms is not None:           
            img = self.transforms(img)   

        label_onehot = nn.functional.label_smooth(label_onehot)    
        return img, label_onehot    
    
    def __len__(self):        
        return len(self.img_path) 